<a href="https://colab.research.google.com/github/Inyrkz/Politician-db/blob/master/Politician.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Inyrkz/Politician-db.git

Cloning into 'Politician-db'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 84 (delta 1), reused 0 (delta 0), pack-reused 78
Unpacking objects: 100% (84/84), done.


In [52]:
import pandas as pd
import string
METADATA = pd.read_csv('/content/Politician-db/Politician DS.csv', encoding = 'latin-1')
METADATA.head(3)

,S/N,Name,Position,Political Party,Elected,Posts
0,1,Okezie Ikpeazu,Current Governor of Abia State,PDP,2019,"Okezie was born on October 18, 1964, to the fa..."
1,2,Ahmadu Umaru Fintiri,Current Governor of Adamawa State,PDP,2019,Ahmadu Umaru Fintiri is the current governor o...
2,3,Udom Gabriel Emmanuel,Current Governor of Akwa-Ibom State,PDP,2019,"Udom Gabriel Emmanuel was born on July 11, 196..."


In [60]:
#Removing NaN values
METADATA = METADATA.fillna('')

#Convert the post elected column to string
METADATA["Elected"] = METADATA["Elected"].astype(int).astype(str)
# METADATA["Posts"] =  METADATA["Posts"].astype(str)
#Joining the other columns into one 
METADATA["General"] = METADATA["Position"] + " " + METADATA["Political Party"] + " " +  METADATA["Elected"]+ " " +  METADATA["Posts"]
METADATA["General"].head()

0    Current Governor of Abia State PDP 2019 Okezie...
1    Current Governor of Adamawa State PDP 2019 Ahm...
2    Current Governor of Akwa-Ibom State PDP 2019 U...
3    Current Governor of Anambra State APGA 2014 Ch...
4    Deputy Governor of Abia State PDP 2019 Rt. Hon...
Name: General, dtype: object

In [61]:
#Importing Libraries
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer object. Remove all english stopwords
TFIDF = TfidfVectorizer(stop_words = 'english')

#Construct the required TF-IDF matrix by fitting and transforming the data
TFIDF_MATRIX = TFIDF.fit_transform(METADATA["General"])

#Let's see the output shape of the matrix
TFIDF_MATRIX.shape
print(TFIDF_MATRIX)

  (0, 2343)	0.06524287832158368
  (0, 870)	0.06524287832158368
  (0, 1730)	0.034136650888568
  (0, 2212)	0.09069102508048156
  (0, 1889)	0.05312131337204646
  (0, 781)	0.059879000382262494
  (0, 2163)	0.06524287832158368
  (0, 1829)	0.05070939047956196
  (0, 794)	0.06524287832158368
  (0, 997)	0.06524287832158368
  (0, 1209)	0.05312131337204646
  (0, 952)	0.059879000382262494
  (0, 306)	0.06524287832158368
  (0, 1633)	0.04395170346934593
  (0, 2304)	0.06524287832158368
  (0, 1678)	0.06524287832158368
  (0, 215)	0.05070939047956196
  (0, 966)	0.0771756510600495
  (0, 199)	0.06524287832158368
  (0, 1652)	0.059879000382262494
  (0, 1298)	0.05070939047956196
  (0, 100)	0.04395170346934593
  (0, 1295)	0.048670138730589715
  (0, 115)	0.05312131337204646
  (0, 113)	0.05312131337204646
  :	:
  (100, 1947)	0.07405752094533476
  (100, 214)	0.08083052319761397
  (100, 120)	0.06010484698258344
  (100, 2312)	0.19487792219775113
  (100, 1814)	0.11483386603904874
  (100, 1076)	0.08532228094502702
  (

In [62]:
#Importing liinear_kernel
from sklearn.metrics.pairwise import linear_kernel

#computing the cosine similarity matrix
COSINE_SIM = linear_kernel(TFIDF_MATRIX, TFIDF_MATRIX)
print(COSINE_SIM)
#Construct a reverse map of the indices and name
indices = pd.Series(METADATA.index, index = METADATA['Name']).drop_duplicates()

[[1.         0.0758485  0.06626482 ... 0.13026969 0.07103056 0.03802277]
 [0.0758485  1.         0.08620109 ... 0.04366637 0.08264922 0.03223337]
 [0.06626482 0.08620109 1.         ... 0.07271414 0.0681688  0.06680664]
 ...
 [0.13026969 0.04366637 0.07271414 ... 1.         0.10376198 0.11090664]
 [0.07103056 0.08264922 0.0681688  ... 0.10376198 1.         0.11277637]
 [0.03802277 0.03223337 0.06680664 ... 0.11090664 0.11277637 1.        ]]


In [0]:

def get_recommendations(name, COSINE_SIM = COSINE_SIM):
  '''Function that takes in a politician's name as input and output similar politicians'''

  #check if name is in database first
  if METADATA['Name'].isin([name]).any() == True:
      #Get the index of the name
      idx = indices[name]

      #Get the pairwise similarity score of all names with that name
      sim_scores = list(enumerate(COSINE_SIM[idx]))

      #sort the names based on similarity scores
      sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

      #Get the scores of the 10 most similar politicians
      sim_scores = sim_scores[1:11]

      #Get the names
      names = [i[0] for i in sim_scores] 
      
      #Return the top 10 similar politicians
      recommendation = list(METADATA['Name'].iloc[names].values[:])
      # return recommendation 
      
      #view the simscore of each prediction
      score = [i[1] for i in sim_scores] #get a list of just the sim score in the list of tuples of index and sim score
      result = pd.Series(score, index=recommendation)
      return result #returns a series of the recommendations as index and the sim score as data
            
  else:
      return  "Name not found in database"
      #if name is not in dataset, return the above

In [66]:
PNAME = input("Search Politician's name: ")
get_recommendations(PNAME)

Search Politician's name: Udom Gabriel Emmanuel


Godswill Akpabio         0.426678
Victor Attah             0.410632
Moses Ekpo               0.373507
Samuel Ortom             0.152975
Baba Tela                0.142792
Willie Obiano            0.125861
Muhammad Inuwa Yahaya    0.117775
Chukwuemeka Ihedioha     0.111688
Benson Abounu            0.111449
Ifeanyi Okowa            0.109614
dtype: float64